Invariant Representations Tutorial
----

This is part of an Invariant Representations tutorial series. There's a short intro here to the theory behind what's going on, but for more in depth coverage see [this unimplemented second part](sorry dead link right now). This tutorial requires basic variational auto-encoder knowledge, and some information theory knowledge, but otherwise should hopefully be pretty accessible.

If you're not familiar with VAE, there's a great tutorial from Lilian Weng [here](https://lilianweng.github.io/lil-log/2018/08/12/from-autoencoder-to-beta-vae.html).
I should find a IT for ML tutorial, but I haven't yet, so if you know of a good one please link me!

PLEASE leave comments/advice/requests. I'm new at this, so this is probably sub-optimal, whatever its current state. I'm also not a regular .ipynb user, so even minor things can be PR'd in happily.

This tutorial is done in Keras/TF.


Programming Setup
----

Okay, so we'll need Keras, Numpy, and MNIST data. I'm going to hide all that, but we're going to get $x$ as a big flat vector and $y$ as a one-hot categorical variable.

In [1]:
import keras
import keras.backend as K
import tensorflow as tf
import numpy as np

IMG_DIM = 28
NUM_LABELS = 10

#data comes as images and 1-dim {0,...,9} categorical variable
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
  
#cast and flatten images, renormalizing to [0,1]
train_x = train_x.astype(np.float32).reshape( (train_x.shape[0], IMG_DIM**2) ) / 255.0
test_x = test_x.astype(np.float32).reshape( (test_x.shape[0], IMG_DIM**2) ) / 255.0

#copypaste
def one_hot(labels):
    num_labels_data = labels.shape[0]
    one_hot_encoding = np.zeros((num_labels_data,NUM_LABELS))
    one_hot_encoding[np.arange(num_labels_data),labels] = 1
    one_hot_encoding = np.reshape(one_hot_encoding, [-1, NUM_LABELS])
    return one_hot_encoding

train_y = one_hot(train_y).astype(np.float32)
test_y = one_hot(test_y).astype(np.float32)

Using TensorFlow backend.


In [2]:
print(train_x.shape)
print(train_y.shape)

(60000, 784)
(60000, 10)


Building a Conditional VAE architecture 
----
<a id="arch-construction"></a>

The Conditional VAE is, as its name suggests, a variational auto-encoder with conditional output. This means it should 
1. Take in $x$ and map it to a latent variable $z$ using the encoder
2. Map $z$ to a new $\hat{x}$ using the decoder.
3. Condition (a.k.a. control) the output $\hat{x}$ by another input $c$, representing specific other factors.

In order to do this we'll also set a few constants. We're using ```DIM_Z=2``` for visualization, but feel free to come back and play around with it later. We're also using "tanh" activations, but you can come back and change this as well (e.g. to "relu").

For some this is pretty basic stuff. Skip to [Loss Construction](#loss-construction) if you can do this on your own.

In [3]:
DIM_Z = 2
DIM_C = NUM_LABELS
INPUT_SHAPE = IMG_DIM ** 2
ACTIVATION = "tanh"

Then we'll build the encoder, which is a two layer fully-connected feed-forward network with two outputs, $z_{mean}$ and $z_{sigma}$. To avoid domain problems, we'll actually output $\log z_{sigma}$. This means we can use a linear layer instead of having to choose a non-negative activation.

In [4]:
#declare inputs to the encoder, which is just x
input_x = keras.layers.Input( shape = [INPUT_SHAPE], name="x" )

#first hidden layer
enc_hidden_1 = keras.layers.Dense(512, activation=ACTIVATION, name="enc_h1")(input_x)
#second hidden layer
enc_hidden_2 = keras.layers.Dense(512, activation=ACTIVATION, name="enc_h2")(enc_hidden_1)

#first output, z_mean
z_mean = keras.layers.Dense(DIM_Z, activation=ACTIVATION)(enc_hidden_2)
#second hidden output, z_log_sigma_sq.
z_log_sigma_sq = keras.layers.Dense(DIM_Z, activation="linear")(enc_hidden_2)

W1203 15:16:10.377006 140196696553280 deprecation_wrapper.py:119] From /data/vision/polina/shared_software/anaconda3_2019.07_py37/envs/dmoyer-default/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1203 15:16:10.394088 140196696553280 deprecation_wrapper.py:119] From /data/vision/polina/shared_software/anaconda3_2019.07_py37/envs/dmoyer-default/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1203 15:16:10.419472 140196696553280 deprecation_wrapper.py:119] From /data/vision/polina/shared_software/anaconda3_2019.07_py37/envs/dmoyer-default/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Creating the latent variable $z$ using a Gaussian layer can be a bit tricky, but luckily is covered in the [Keras documentation](https://keras.io/examples/variational_autoencoder/), which this part follows almost exactly. If you haven't seen the reparameterization trick before, this is to create a Gaussian distributed layer $z$ which can be differentiated w.r.t. its parameters. It was popularized by [Kingma and Welling 2014](link here). 

In [5]:
#stolen straight from the docs
#https://keras.io/examples/variational_autoencoder/
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


z_mean = keras.layers.Dense(DIM_Z, activation="tanh")(enc_hidden_2)
z_log_sigma_sq = keras.layers.Dense(DIM_Z, activation="linear")(enc_hidden_2)

z = keras.layers.Lambda(sampling, output_shape=(DIM_Z,), name='z')([z_mean, z_log_sigma_sq])

W1203 15:16:14.491023 140196696553280 deprecation_wrapper.py:119] From /data/vision/polina/shared_software/anaconda3_2019.07_py37/envs/dmoyer-default/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.



Great, that's the Encoder from $x$ to $z$ done. Let's build the conditional decoder. There isn't an established standard way to condition the output, but concatenating $z$ and $c$ before inputting into the decoder is good enough for now.

In [6]:
#declare any additional inputs to our decoder, in this case c
input_c = keras.layers.Input( shape = [DIM_C], name="c")
#this is the concat operation!
z_with_c = keras.layers.concatenate([z,input_c])

#first hidden layer
dec_hidden_1 = keras.layers.Dense(512, activation=ACTIVATION, name="dec_h1")(z_with_c)
#second hidden layer
dec_hidden_2 = keras.layers.Dense(512, activation=ACTIVATION, name="dec_h2")(dec_hidden_1)

#output, should be same domain as x_hat
#could also use sigmoid activation
x_hat = keras.layers.Dense( INPUT_SHAPE, name="x_hat", activation="linear" )(dec_hidden_2)

Loss Construction for Invariance (...and also all that VAE stuff)
----
<a id="loss-construction"></a>

Okay, so we have both the encoder and the conditional decoder now, so the next step is building the loss function. There are three sub-components:

1. Reconstruction (how far is $\hat{x}$ from $x$), usually $\|x - \hat{x}\|_2^2$.
2. "Distance" to the Prior (from the original VAE definition), $KL[q(z|x)| p(z)]$
3. "Distance" to the Empirical Marginal Posterior (from our paper, among others), $KL[q(z|x)| q(z)]$.

The third one we'll have to approximate, so we'll deal with that second. First, the two easy ones, straight from Keras Docs, plus defining some hyper parameters:

In [8]:
params = {
  "beta" : 0.1,
  "lambda" : 1.0,
}

recon_loss = keras.losses.mse(input_x, x_hat)
recon_loss *= INPUT_SHAPE #optional, in the tutorial code though

prior_loss = 1 + z_log_sigma_sq - K.square(z_mean) - K.exp(z_log_sigma_sq)
prior_loss = K.sum(prior_loss, axis=-1)
prior_loss *= -0.5

Now for $KL[ q(z|x) | q(z) ]$. Since we're using the Gaussian $z$ layer, there's an approximation we can make using the pairwise Gaussian KL divergences. **In the original version of the paper there is an erroneous extra term at this part.** The corrected version is implemented below, but before we get into it, we should remember that we actually want to compute $KL[ q(z|x) | q(z) ]$, not this bound, and that this could also be solved using:

* Direct approximation (e.g. use a neural network to approximate posterior marginal $q(z)$ term given $q(z|x)$ parameters and samples $x$).
* Sampling
* Use a different $z$ layer with analytic divergence to its marginal.

The first and second options appear in the literature; see e.g. [Fixing a Broken ELBO (Alemi et al. 2017)](https://arxiv.org/abs/1711.00464) and [Structured Disentangled Representations Esmaeili et al. 2018](https://arxiv.org/abs/1804.02086). The third option appears in our paper [Echo Noise](https://arxiv.org/abs/1904.07199), which we'll demonstrate at the end.

Knowing this, here's one way to do it *for Gaussian $z$ layers* using pairwise KL divergence:

In [9]:

#KL(N_0|N_1) = tr(\sigma_1^{-1} \sigma_0) + 
#  (\mu_1 - \mu_0)\sigma_1^{-1}(\mu_1 - \mu_0) - k +
#  \log( \frac{\det \sigma_1}{\det \sigma_0} )
def all_pairs_gaussian_kl(mu, sigma, add_third_term=False):
    sigma_sq = tf.square(sigma) + 1e-8
    sigma_sq_inv = tf.reciprocal(sigma_sq)

    #dot product of all sigma_inv vectors with sigma is the same as a matrix mult of diag
    first_term = tf.matmul(sigma_sq,tf.transpose(sigma_sq_inv))
    
    r = tf.matmul(mu * mu,tf.transpose(sigma_sq_inv))
    r2 = mu * mu * sigma_sq_inv 
    r2 = tf.reduce_sum(r2,1)
 
    #squared distance
    #(mu[i] - mu[j])\sigma_inv(mu[i] - mu[j]) = r[i] - 2*mu[i]*mu[j] + r[j]
    #uses broadcasting
    second_term = 2*tf.matmul(mu, tf.transpose(mu*sigma_sq_inv))
    second_term = r - second_term + tf.transpose(r2)

    # log det A = tr log A
    # log \frac{ det \Sigma_1 }{ det \Sigma_0 } =
    #   \tr\log \Sigma_1 - \tr\log \Sigma_0 
    # for each sample, we have B comparisons to B other samples...
    #   so this cancels out

    if(add_third_term):
        r = tf.reduce_sum(tf.log(sigma_sq),1)
        r = tf.reshape(r,[-1,1])
        third_term = r - tf.transpose(r)
    else:
        third_term = 0

    #- tf.reduce_sum(tf.log(1e-8 + tf.square(sigma)))\
    # the dim_z ** 3 term comes fro
    #   -the k in the original expression
    #   -this happening k times in for each sample
    #   -this happening for k samples
    #return 0.5 * ( first_term + second_term + third_term - dim_z )
    return 0.5 * ( first_term + second_term + third_term )

#
# kl_conditional_and_marg
#   \sum_{x'} KL[ q(z|x) \| q(z|x') ] + (B-1) H[q(z|x)]
#

#def kl_conditional_and_marg(args):
def kl_conditional_and_marg(z_mean, z_log_sigma_sq, dim_z):
    z_sigma = tf.exp( 0.5 * z_log_sigma_sq )
    all_pairs_GKL = all_pairs_gaussian_kl(z_mean, z_sigma, True) - 0.5*dim_z
    return tf.reduce_mean(all_pairs_GKL)

So after all that, we can create our third loss term. We'll then add them all up, and create our model!

In [12]:
kl_qzx_qz_loss = kl_conditional_and_marg(z_mean, z_log_sigma_sq, DIM_Z)

#Invariant Conditional Variational Autoencoder (ICVAE).
#I think the name game is meh (what if someone else got there first? or defines a different one later?)
#so maybe it'd be easier to say Moyer 2018? too narcissistic? Conditional VAE with additional compression?
#
# ...the point is, this one can induce invariance. So can the others sometimes, too, in practice.

icvae_loss = K.mean((1 + params["lambda"]) * recon_loss + params["beta"]*prior_loss + params["lambda"]*kl_qzx_qz_loss)

icvae = keras.models.Model(inputs=[input_x,input_c], outputs=x_hat, name="ICVAE")

icvae.add_loss(icvae_loss)

learning_rate = 0.0005
opt = keras.optimizers.Adam(lr=learning_rate)

icvae.compile( optimizer=opt, )


W1203 15:17:10.308701 140196696553280 deprecation_wrapper.py:119] From /data/vision/polina/shared_software/anaconda3_2019.07_py37/envs/dmoyer-default/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



## Training

Only run this bit once. It takes a bit (but not too long tbh). If this isn't your jupyter server, you should delete the path stuff. I just wanted to save you from repetitive gpu time/coffee breaks.

In [ ]:
import os
if not os.path.exists("mnist_icvae.h5"):
  cvae.fit(
    { "x" : train_x, "c" : train_y }, epochs=100
  )
  cvae.save_weights("mnist_icvae.h5")
else:
  cvae.load_weights("mnist_icvae.h5")

# Plots and Evaluation!

Okay, so now that we've trained it, let's see how well it does!

In [ ]:
n_plot_samps = 10
test_x_hat = mean_cvae.predict(
 { "x" : test_x[:n_plot_samps], "c" : test_y[:n_plot_samps] }
)

##
## plot first N
##

from plot_tools import pics_tools as pic
import matplotlib.pyplot as plt

fig = pic.plot_image_grid( \
  np.concatenate([test_x[:n_plot_samps],test_x_hat], axis=0),
  [mnist_dataset.IMG_DIM, mnist_dataset.IMG_DIM], \
  (2,n_plot_samps) \
)
plt.show()